In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import copy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import math

In [ ]:
file = files.upload()

In [ ]:

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from google.colab import drive
drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

with open('/content/drive/MyDrive/Colab Notebooks/Brown_train.txt','r') as file:
  data = file.readlines()
  # data = file.read()

In [ ]:

# tokens = nltk.word_tokenize(data)
# print(tokens[:3])

tokens = []
for i in data:
  tokens.append([tuple(j.split("/")) for j in i.split()])
# for i in data:
#   tokens.append([j for j in i.split()])
  # tokens.extend(i.split())

l1 = tokens
for i in l1:
  for j in i:
    if(len(j)>2):
      i.append((j[0],j[-1]))
      i.remove(j)

tokens = l1
# l1=[]
# for i in tokens:
#   for j in i:
#     if(len(j)>2):
#       l1.append(j)
# print(l1)
print(tokens[:3])

from sklearn.model_selection import train_test_split
# split data into training and test set in the ratio 80:20
train_set,test_set = train_test_split(tokens,train_size=0.80,test_size=0.20,random_state = None)


[[('At', 'ADP'), ('that', 'DET'), ('time', 'NOUN'), ('highway', 'NOUN'), ('engineers', 'NOUN'), ('traveled', 'VERB'), ('rough', 'ADJ'), ('and', 'CONJ'), ('dirty', 'ADJ'), ('roads', 'NOUN'), ('to', 'PRT'), ('accomplish', 'VERB'), ('their', 'DET'), ('duties', 'NOUN'), ('.', '.')], [('Using', 'VERB'), ('privately-owned', 'ADJ'), ('vehicles', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), ('personal', 'ADJ'), ('hardship', 'NOUN'), ('for', 'ADP'), ('such', 'ADJ'), ('employees', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('the', 'DET'), ('matter', 'NOUN'), ('of', 'ADP'), ('providing', 'VERB'), ('state', 'NOUN'), ('transportation', 'NOUN'), ('was', 'VERB'), ('felt', 'VERB'), ('perfectly', 'ADV'), ('justifiable', 'ADJ'), ('.', '.')], [('Once', 'ADP'), ('the', 'DET'), ('principle', 'NOUN'), ('was', 'VERB'), ('established', 'VERB'), (',', '.'), ('the', 'DET'), ('increase', 'NOUN'), ('in', 'ADP'), ('state-owned', 'ADJ'), ('vehicles', 'NOUN'), ('came', 'VERB'), ('rapidly', 'ADV'), ('.', '.')]]


In [ ]:
tagged_data=[]
for i in train_set:
  tagged_data.extend(i)
print(tagged_data[:3])

[('To', 'ADP'), ('his', 'DET'), ('puzzlement', 'NOUN')]


In [ ]:
class HMM:
    def __init__(self, order):
        self.order = order
        self.states = set()
        self.start_prob = defaultdict(float)
        self.transition_prob = defaultdict(lambda: defaultdict(float))
        self.emission_prob = defaultdict(lambda: defaultdict(float))

    def train(self, dataset):
        tag_count = defaultdict(int)
        tag_pair_count = defaultdict(lambda: defaultdict(int))
        tag_triple_count = defaultdict(lambda: defaultdict(int))
        tag_word_count = defaultdict(lambda: defaultdict(int))

        for sentence in dataset:
            prev_tags = ['<s>'] * self.order
            for word, tag in sentence:
                tag_count[tag] += 1
                for i in range(1, self.order + 1):
                    if i == 1:
                        tag_pair_count[prev_tags[-1]][tag] += 1
                    else:
                        tag_triple_count[tuple(prev_tags[-i:])][tag] += 1
                tag_word_count[tag][word] += 1
                self.states.add(tag)
                prev_tags.append(tag)
                prev_tags.pop(0)

        total_tags = sum(tag_count.values())

        for tag, count in tag_count.items():
            self.start_prob[tag] = count / total_tags

        for prev_tag, next_tags in tag_pair_count.items():
            total_transitions = sum(next_tags.values())
            for next_tag, count in next_tags.items():
                self.transition_prob[prev_tag][next_tag] = count / total_transitions

        for prev_tags, next_tags in tag_triple_count.items():
            total_transitions = sum(next_tags.values())
            for next_tag, count in next_tags.items():
                self.transition_prob[prev_tags][next_tag] = count / total_transitions

        for tag, word_counts in tag_word_count.items():
            total_tag = tag_count[tag]
            for word, count in word_counts.items():
                self.emission_prob[tag][word] = count / total_tag

    def tag(self, sequence):
        V = [{}]
        path = {}

        for state in self.states:
            if sequence[0] in self.emission_prob[state]:
                V[0][state] = self.start_prob[state] * self.emission_prob[state][sequence[0]]
            else:
                V[0][state] = 0
            path[state] = [state]

        for t in range(1, len(sequence)):
            V.append({})
            new_path = {}

            for state in self.states:
                if sequence[t] in self.emission_prob[state]:
                    (prob, prev_state) = max((V[t - 1][prev_state] * self.transition_prob[prev_state][state] * self.emission_prob[state][sequence[t]], prev_state) for prev_state in self.states)
                    V[t][state] = prob
                    new_path[state] = path[prev_state] + [state]
                else:
                    V[t][state] = 0
            path = new_path

        (prob, state) = max((V[len(sequence) - 1][final_state], final_state) for final_state in self.states)
        return path[state]


    def viterbi(self, observations):
        # Initialize variables
        viterbi = [{state: {"prob": 0, "prev_state": None} for state in self.states} for _ in range(len(observations))]
        # Initialize starting probabilities
        for state in self.states:
            viterbi[0][state]["prob"] = self.start_prob[state] * self.emission_prob[state][observations[0]]
            viterbi[0][state]["prev_state"] = None

        # Forward pass
        for t in range(1, len(observations)):
            for state in self.states:
                max_prob = 0
                max_prev_state = None
                for prev_state in self.states:
                    prob = viterbi[t-1][prev_state]["prob"] * self.transition_prob[prev_state][state] * self.emission_prob[state][observations[t]]
                    if prob > max_prob:
                        max_prob = prob
                        max_prev_state = prev_state
                viterbi[t][state]["prob"] = max_prob
                viterbi[t][state]["prev_state"] = max_prev_state

        # Backtrack to find the best path
        best_path = []
        max_prob = 0
        max_state = None
        for state in self.states:
            if viterbi[-1][state]["prob"] > max_prob:
                max_prob = viterbi[-1][state]["prob"]
                max_state = state
        best_path.append(max_state)
        prev_state = max_state
        for t in range(len(observations)-1, 0, -1):
            prev_state = viterbi[t][prev_state]["prev_state"]
            best_path.insert(0, prev_state)

        return best_path

In [ ]:
hmm_bi = HMM(order=1)
hmm_tri = HMM(order=2)

hmm_bi.train(tokens)
hmm_tri.train(tokens)



In [ ]:
print(hmm_bi.states)
print(hmm_bi.start_prob)
print(hmm_bi.emission_prob)
print(hmm_bi.transition_prob)
print("\n")
# print("\n")
print("\n")
print(hmm_tri.states)
print(hmm_tri.start_prob)
print(hmm_tri.emission_prob)
print(hmm_tri.transition_prob)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(test_set[0])

for i in test_set:
  sentence = []
  tags = []
  for j in i:
    sentence.append(j[0])
    tags.append(j[1])
  # print(hmm_bi.viterbi(sentence))
  # print(hmm_tri.viterbi(sentence))

# sentence = []
# tags = []
# for i in test_set[0]:
#   # for j in i:
#   sentence.append(i[0])
#   tags.append(i[1])

# print(hmm_bi.viterbi(sentence))
# print(hmm_tri.viterbi(sentence))

# print(tags)

  y_test_fold = hmm_bi.viterbi(sentence)
  y_pred_fold = tags

  accuracy_fold = accuracy_score(y_test_fold, y_pred_fold)
  precision_fold = precision_score(y_test_fold, y_pred_fold, average='weighted')
  recall_fold = recall_score(y_test_fold, y_pred_fold, average='weighted')
  f1_fold = f1_score(y_test_fold, y_pred_fold, average='weighted')

  print(f1_fold)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Assuming you have a model named 'your_model' and data 'X_train', 'y_train'
# X_train is your feature matrix, and y_train is your target variable

X_train = train_set
y_train = test_set

# Initialize metrics lists for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

# Initialize StratifiedKFold with 5 folds
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate through each fold
for train_index, test_index in kf.split(X_train, y_train):
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    # Assuming 'your_model' is a classifier, you need to train it on the training fold
    your_model.fit(X_train_fold, y_train_fold)

    # Predict on the test fold
    y_pred_fold = your_model.predict(X_test_fold)

    # Calculate metrics for this fold
    accuracy_fold = accuracy_score(y_test_fold, y_pred_fold)
    precision_fold = precision_score(y_test_fold, y_pred_fold, average='weighted')
    recall_fold = recall_score(y_test_fold, y_pred_fold, average='weighted')
    f1_fold = f1_score(y_test_fold, y_pred_fold, average='weighted')

    # Append metrics to lists
    accuracy_list.append(accuracy_fold)
    precision_list.append(precision_fold)
    recall_list.append(recall_fold)
    f1_list.append(f1_fold)

# Calculate average metrics across all folds
average_accuracy = np.mean(accuracy_list)
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)
average_f1 = np.mean(f1_list)

# Print individual fold results
for i in range(5):
    print(f"Fold {i + 1}: Accuracy={accuracy_list[i]}, Precision={precision_list[i]}, Recall={recall_list[i]}, F1-Score={f1_list[i]}")

# Print average results
print("\nAverage Results:")
print(f"Average Accuracy: {average_accuracy}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

# *RNN*

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed

In [ ]:
def read_data(file_path):
    sentences = []
    tags = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip().split()
            sentence = []
            tag = []
            for word_tag_pair in line:
                # print(word_tag_pair)
                if len(word_tag_pair.split('/'))== 2:
                  word, tag_label = word_tag_pair.split('/')
                  sentence.append(word)
                  tag.append(tag_label)
            sentences.append(sentence)
            tags.append(tag)
    return sentences, tags

file_path = '/content/Brown_train.txt'
sentences, tags = read_data(file_path)

In [ ]:
rnn_dataset = []
for i,j in list(zip(sentences,tags)):
  temp = (" ".join(i),j)
  rnn_dataset.append(temp)

In [ ]:
import tensorflow as tf
import numpy as np


# Hyperparameters
EMBEDDING_DIM = 50
HIDDEN_DIM = 50
LEARNING_RATE = 0.01
NUM_EPOCHS = 5

training_data = rnn_dataset
# Create vocabulary and POS tag sets
word_to_ix = {}
tag_to_ix = {}
for sentence, tags in training_data:
    for word in sentence.split():
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

# Convert training data to numerical format
X_train = []
y_train = []
for sentence, tags in training_data:
    X_train.append([word_to_ix[word] for word in sentence.split()])
    y_train.append([tag_to_ix[tag] for tag in tags])

# Pad sequences to make them the same length
MAX_LEN = max(len(sentence) for sentence in X_train)
X_train = pad_sequences(X_train, maxlen=MAX_LEN, padding='post')
y_train = pad_sequences(y_train, maxlen=MAX_LEN, padding='post')

# Model definition
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_to_ix), EMBEDDING_DIM),
    tf.keras.layers.SimpleRNN(HIDDEN_DIM, return_sequences=True),
    tf.keras.layers.Dense(len(tag_to_ix), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=NUM_EPOCHS)

In [ ]:
# Example inference
sentence = "At that time highway engineers traveled rough and dirty roads to accomplish their duties"
input_sentence = [word_to_ix[word] for word in sentence.split()]
input_sentence = pad_sequences([input_sentence], maxlen=MAX_LEN, padding='post')
predicted_tags = model.predict(input_sentence)
predicted_tags = [np.argmax(tag) for tag in predicted_tags[0]]
predicted_tags = [list(tag_to_ix.keys())[list(tag_to_ix.values()).index(idx)] for idx in predicted_tags]
print("Predicted POS tags:", predicted_tags[:len(sentence.split())])